# Gradient Descent

In [115]:
import tensorflow as tf
import numpy as np

In [3]:
tf.__version__

'2.0.0'

경사하강법 : Gradient Descent Optimization  
경사 : 미분하면 경사를 구할 수 있다.(기울기)

In [4]:
x = [1,2,3,4,5.0] # 실수로 만들기 위해 .0을 하나 붙입니다.
y = [3,5,7,9,11.0]
# y = 2 * x + 1 -> y = W * x + b
W = tf.Variable(1.0) # 변수 : 학습되는 파라미터, 초기값 1.0
b = tf.Variable(0.0) # 초기값 0.0

h = W * x + b # Hyperthesis 가설, 예측
h

<tf.Tensor: id=19, shape=(5,), dtype=float32, numpy=array([1., 2., 3., 4., 5.], dtype=float32)>

$x$와 $y$가 주어졌을 때 $ h = W \times X + b $의 계산식을 완성하는 $W$와 $b$의 값을 찾아보자

In [6]:
# 오차 측정
cost = tf.reduce_mean(tf.square(h - y)) # Mean 평균, Square 제곱 Error 오차
cost

<tf.Tensor: id=24, shape=(), dtype=float32, numpy=18.0>

W와 b값의 변화를 주어 cost(오차)를 줄인다.

In [7]:
W = tf.Variable(2.0)
b = tf.Variable(1.0)
h = W * x + b
cost = tf.reduce_mean(tf.square(h - y))
cost

<tf.Tensor: id=48, shape=(), dtype=float32, numpy=0.0>

In [10]:
W = tf.Variable(5.0)
b = tf.Variable(0.0)
with tf.GradientTape() as tape: # 경사 기록 장치: 이 안에서 수행되는 연산의 경사가 기록됩니다.
    h = W * x + b
    cost = tf.reduce_mean(tf.square(h-y))
w_grad = tape.gradient(cost, [W])
w_grad

[<tf.Tensor: id=171, shape=(), dtype=float32, numpy=60.0>]

W의 초기값을 1.0으로 잡게 되면 경사값은 -28.0  
W의 초기값을 5.0으로 잡게 되면 경사값은 60.0  
이 나오게 되는데 이 말은 $W$의 값을 경사값만큼 빼라는 의미이다.  
**그러나** 이 값을 그대로 빼게 되면 너무도 변화량이 커지며 값도 커지기 때문에 학습률이라는 것을 곱한 결과값을 빼야 한다.

In [107]:
W = tf.Variable(10.0)
b = tf.Variable(0.0)

In [100]:
with tf.GradientTape() as tape: # 경사 기록 장치: 이 안에서 수행되는 연산의 경사가 기록됩니다.
    h = W * x + b
    cost = tf.reduce_mean(tf.square(h-y))
w_grad = tape.gradient(cost, [W])
learning_rate = 0.01
W.assign_sub(learning_rate * w_grad[0])
W, w_grad

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.2727277>,
 [<tf.Tensor: id=3648, shape=(), dtype=float32, numpy=1.0251999e-05>])

위의 코드를 반복하는 과정을 통해 초기값이 5.0이었던 $W$의 값이 정답인 2.0에 가까워지는 것을 발견할 수 있다.

In [108]:
for i in range(1000):
    with tf.GradientTape() as tape: # 경사 기록 장치: 이 안에서 수행되는 연산의 경사가 기록됩니다.
        h = W * x + b
        cost = tf.reduce_mean(tf.square(h-y))
    w_grad, b_grad = tape.gradient(cost, [W, b])
    learning_rate = 0.01
    W.assign_sub(learning_rate * w_grad)
    b.assign_sub(learning_rate * b_grad)
W, b

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.027979>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.8989878>)

1000번 반복한 결과 $W$는 2에 $b$는 1에 가까워진 것을 알 수 있다.(초기값: (10, 0))

In [148]:
x = [1,2,3,4,5.0]
y = [3,5,7,9,11.0]

`model = keras.Sequencial()`의 경우 기본적, 기초적인 사용법이라 커스터마이징이 어렵습니다.

In [149]:
import tensorflow as tf
from tensorflow import keras

class SimpleModel(tf.keras.Model):
    def __init__(self):
        super(SimpleModel, self).__init__()
        print("init")
        self.dense_0 = keras.layers.Dense(1) # dense: 완전연결
        # layer를 추가하고 싶은 경우 바로 위의 코드와 같이 이곳에 작성
        
    def call(self, x):
        h = self.dense_0(x)
        h = tf.squeeze(h, axis=1) # (m,1)을 (m)으로 차원축소합니다.
        return h
    
model = SimpleModel()
optimizer = tf.optimizers.SGD(learning_rate=0.07)

def loss(y, h):
    return tf.reduce_mean(tf.square(y-h))
def update(x, y):
    with tf.GradientTape() as tape:
        h = model(x)
        cost = loss(y, h)
    grads = tape.gradient(cost, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return cost

init


In [150]:
x_2d = np.array(x).reshape([-1,1]).astype(np.float32)
y = np.array(y).astype(np.float32)
for i in range(200):
    update(x_2d, y)
print('finish', model.trainable_variables) # 값 확인

finish [<tf.Variable 'simple_model_13/dense_11/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[2.0009394]], dtype=float32)>, <tf.Variable 'simple_model_13/dense_11/bias:0' shape=(1,) dtype=float32, numpy=array([0.99660903], dtype=float32)>]


약 200번 반복한 결과 $W$는 2.0009394로 2에 아주 가까운 수가 $b$는 0.99660903으로 1에 아주 가까운 수가 나오는 것을 확인할 수 있다.